https://edition.cnn.com/


https://www.cnbc.com/weather-and-natural-disasters/
https://www.euronews.com/tag/natural-disaster

https://www.goodnewsnetwork.org/

<ul>
    <li>Check which news provider is more neutral, negative and Positive</li>

</ul>
<h4>Vader</h4>
<p>Limits to consider neutral positive and negative</p>
<h4>TextBlob</h4>


<h4>Flair</h4>

In [89]:
# 
import os
import csv
import time
# importing libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os


# Selenium
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser


#nlp
import nltk
from nltk.corpus import stopwords
from string import punctuation
import unicodedata
import contractions
STOPWORDS = set(stopwords.words('english'))


#regular expressions
import re

# pretrained (sentiment classification)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# textBlob 
from textblob import TextBlob

# flair
import flair

# loop status viewer
from tqdm import tqdm

# tools
import matplotlib.pyplot as plt
import random


# train test split
from sklearn.model_selection import train_test_split


# tensorflow libraries to train new classification model
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

# parallel computing
from joblib import parallel_backend


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/salvado/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [90]:
# set the driver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [91]:
def goodnews_scrap(web='https://www.goodnewsnetwork.org/', pages=1)-> list:
    driver.get(web)
    
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 2
    article_goodnews = []
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(3)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        #if (screen_height) * i > scroll_height:
        #    break
        print(i)
        
        soup_goodnews = BeautifulSoup(driver.page_source)
        
        news_list = soup_goodnews.find_all('div', class_='td-block-row')
        for news in news_list:
            for header in news.find_all('h3'):
                if len(header.get_text().split()) > 0:
                    article_goodnews.append(header.get_text())
                    
    return article_goodnews
    

In [103]:
def goodnews_scrap_v2(web='https://www.goodnewsnetwork.org/', n_articles = 10):
    """
        This function is to perform webscrap from the website Goodnews. This website is a website that
        will load more articles when one scrolldown. So we'll use Selenium to help us with that.
        We define also the number of articles we want to scrap.
        
    :param web: Goodnews WebSite 
    :param n_articles:  number of articles to scrap from the website  
    :return: list of headers from each article-
    """
    
    driver.get(web)
    # loading time issues:
    scroll_pause_time = 2 # seconds
    screen_height = driver.execute_script('return window.screen.height;')  # get the height of one screen
    scrolling = True
    # for scrolling purposes
    scroll_init = 0
    i=0
    
    # init list article
    article_title = []
    
    while scrolling:
        driver.execute_script('window.scrollTo({scroll_init}, {screen_height});'.format(scroll_init=scroll_init, screen_height=screen_height))
        i += 1
        scroll_init = screen_height
        screen_height = scroll_init + screen_height
        time.sleep(3)
        
        soup_goodnews = BeautifulSoup(driver.page_source)
        news_list = soup_goodnews.find_all('div', class_='td-block-row')
        for news in news_list:
            for header in news.find_all('h3'):
                if len(header.get_text().split()) > 0:
                    article_title.append(header.get_text())
                    if len(article_title) > n_articles:
                        scrolling = False
                        break
                        
    return article_title

In [102]:
articles_goodnews = goodnews_scrap_v2(n_articles=300)
#articles_cnn = cnn_scrap()

print(f'Goodnews Number of Articles: {len(articles_goodnews)}')
#print(f'CNN Number of Articles: {len(articles_cnn)}')


Goodnews Number of Articles: 324


In [104]:
def cnn_scrap(web='https://edition.cnn.com/'):
    driver.get(web)
    soup_cnn = BeautifulSoup(driver.page_source)
    article_title = []
    for section in soup_cnn.find_all('section')[1:]:
        if len(section.find_all('ul')) > 0:
            for ul_elem in section.find_all('ul'):
                for elem in ul_elem.find_all('li'):
                    if len(elem.get_text().split()) > 0:
                        article_title.append(elem.get_text())
    return article_title

In [105]:
#articles_goodnews = goodnews_scrap()
articles_cnn = cnn_scrap()

#print(f'Goodnews Number of Articles: {len(articles_goodnews)}')
print(f'CNN Number of Articles: {len(articles_cnn)}')


CNN Number of Articles: 0


In [188]:
def cnn_scrapv2(web='https://edition.cnn.com/world'):
    """
        This function is to perform webscrap from the website CNN. This function is going to scrap each "layer",
        from the nav-bar on the top and the page "in it". (Africa, Europe, Americas, etc.)
        
    :param web: CNN World WebSite 
    :return: list of headers from each article-
    """
    driver.get(web)
    cnn_world = BeautifulSoup(driver.page_source)
    
    nav_items = cnn_world.find_all('div', class_='header__nav-item')
    #print(nav_items)
    url_list = []
    for url in nav_items:
        url_list.append(url.find('a')['data-zjs-destination_url'])
    article_title = []    
    for url in url_list:
        driver.get(url)
        page_soup = BeautifulSoup(driver.page_source)
        tmp = page_soup.find_all('section', class_='layout__wrapper')[0]
        for header in tmp.find_all('div', class_='container__headline'):
            if len(header.get_text().split()) > 0:
                article_title.append(header.get_text(strip=True))

    return article_title

<h3>Text preprocessing:</h3>
<ul>
    <li>remove accented characters from strings use UniDecode;  <b>check :D</b></li>
    <li>remove punctuation;  <b>check :D</b></li>
    <li>make everyword lowercase;  <b>check :D</b></li>
    <li>if present: remove the \n \t, etc from the strings;  <b>check :D</b></li>
    <li>try remove stopwords;  <b>check :D</b></li>
</ul>

In [6]:
def text_preprocessing(text: str, stop_word=True)-> str:
    """
    Function to preprocess text.
    """
    tmp = []
    for word in text.split():
        if word in list(contractions.contractions_dict.keys()):
            tmp.append(contractions.contractions_dict[word])
        elif re.findall(r"[A-Za-z]+"r"'"+r"[A-Za-z]", word):
            split_at = re.search(r"[']", word).span()[0]
            word = word[0:split_at]
            tmp.append(word)
        elif re.findall(r"[a-zA-z0-9]+"r"-"+r"[a-zA-z0-9]", word):
            l =re.split(r'-', word)
            tmp.extend(l)
        else:
            tmp.append(word)
    text = ' '.join(tmp)
    text = text.translate(str.maketrans('','', punctuation))  # remove punctutations
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    if stop_word:
        text = [word for word in text.split() if word not in STOPWORDS]
        text = ' '.join(text)
    
    
    
    return text

In [21]:
def classify_text_vader(articles: list, dataframe_is_or_no=True)-> pd.DataFrame:
    DS = SentimentIntensityAnalyzer()
    articles_ = []
    article_class = []
    for article in articles:
        text_original_classified = DS.polarity_scores(article)
        text_processed = text_preprocessing(article, False)
        text_processed_classified = DS.polarity_scores(text_processed)
        if dataframe_is_or_no:
            articles_.append([article, text_original_classified['compound'], text_processed, text_processed_classified['compound']])
        else:
            article_class.append([text_original_classified['compound'], text_processed_classified['compound']])
    if dataframe_is_or_no:        
        return pd.DataFrame(articles_, columns=['Original', 'Original Sent', 'Processed', 'Processed Sent'])
    else:
        return article_class

In [22]:
def classify_text_Texblob(articles: list, dataframe_is_or_no=True)-> pd.DataFrame:
    """
    Function that will classify a sentence.
    
    """
    articles_ = []
    article_class = []
    for article in articles:
        text_original_classified = TextBlob(article).sentiment
        text_processed = text_preprocessing(article, False)
        text_processed_classified = TextBlob(text_processed).sentiment
        if dataframe_is_or_no:
            articles_.append([article, text_original_classified, text_processed, text_processed_classified])
        else:
            article_class.append([text_original_classified[0], text_processed_classified[0]])
        
    if dataframe_is_or_no:        
        return pd.DataFrame(articles_, columns=['Original', 'Original Sent', 'Processed', 'Processed Sent'])
    else:
        return article_class

In [23]:
def clasify_text_flair(articles: list, dataframe_is_or_no=True)-> pd.DataFrame:
    """
    Function that will classify a sentence.
    
    """
    articles_ = []
    article_class = []
    flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
    for article in tqdm(articles):
        text_original_classified = flair.data.Sentence(article)
        flair_sentiment.predict(text_original_classified)
        sentiment_o = text_original_classified.score
        if re.findall('negative', str(text_original_classified.labels[0]), re.IGNORECASE):
            # print('not cool')
            sentiment_o = -1 * float(sentiment_o)
        
        
        
        text_processed = text_preprocessing(article, False)
        text_processed_classified = flair.data.Sentence(text_processed)
        flair_sentiment.predict(text_processed_classified)
        sentiment_p = text_processed_classified.score
        if re.findall('negative', str(text_processed_classified.labels[0]), re.IGNORECASE):
            # print('not cool')
            sentiment_p = -1 * float(sentiment_p)
        
        
        if dataframe_is_or_no:
            articles_.append([article, sentiment_o, text_processed, sentiment_p])
        else:
            article_class.append([sentiment_o, sentiment_p])
        
    if dataframe_is_or_no:        
        return pd.DataFrame(articles_, columns=['Original', 'Original Sent', 'Processed', 'Processed Sent'])
    else:
        return article_class  
    

In [24]:
vader_df = classify_text_vader(articles_goodnews)
textblob_df = classify_text_Texblob(articles_goodnews)
flair_df = clasify_text_flair(articles_goodnews)
vader_df.shape, textblob_df.shape, flair_df.shape

100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 11.71it/s]


((11, 4), (11, 4), (11, 4))

In [25]:
df_dict = {'Original':vader_df['Original'],
          'O_Vader': vader_df['Original Sent'],
          'O_TextBlob': textblob_df['Original Sent'],
          'O_Flair': flair_df['Original Sent'],
          'Processed':vader_df['Processed'], 
          'P_Vader':vader_df['Processed Sent'],
          'P_TextBlob': textblob_df['Processed Sent'],
          'P_Flair': flair_df['Processed Sent']}

In [26]:
df = pd.DataFrame.from_dict(df_dict)
df.head(5)

,Original,O_Vader,O_TextBlob,O_Flair,Processed,P_Vader,P_TextBlob,P_Flair
0,"Good News in History, February 21",0.4404,"(0.7, 0.6000000000000001)",0.990672,good news in history february 21,0.4404,"(0.7, 0.6000000000000001)",0.996886
1,First of Its Kind Discovery in Mali: Vast Rese...,0.7269,"(0.3041666666666667, 0.7333333333333334)",0.996523,first of its kind discovery in mali vast reser...,0.7269,"(0.3041666666666667, 0.7333333333333334)",0.997266
2,Amazing Footage Shows Birth of ‘Precious’ Rare...,0.5859,"(0.46666666666666673, 0.9333333333333332)",0.999854,amazing footage shows birth of precious rare t...,0.8176,"(0.46666666666666673, 0.9333333333333332)",0.999780
3,Astronomers Observe 2 Neutron Stars Colliding ...,-0.1280,"(-0.125, 1.0)",0.867790,astronomers observe 2 neutron stars colliding ...,-0.1280,"(-0.125, 1.0)",0.945756
4,North America’s Only Native Stork Poised to Fl...,0.2500,"(0.4, 0.95)",-0.951812,north americas only native stork poised to fly...,0.2500,"(0.4, 0.95)",-0.835691


In [13]:
# cnn news
vader_df = classify_text_vader(articles_cnn)
textblob_df = classify_text_Texblob(articles_cnn)
flair_df = clasify_text_flair(articles_cnn)
# vader_df.shape, textblob_df.shape, flair_df.shape

df_dict = {'Original':vader_df['Original'],
          'O_Vader': vader_df['Original Sent'],
          'O_TextBlob': textblob_df['Original Sent'],
          'O_Flair': flair_df['Original Sent'],
          'Processed':vader_df['Processed'], 
          'P_Vader':vader_df['Processed Sent'],
          'P_TextBlob': textblob_df['Processed Sent'],
          'P_Flair': flair_df['Processed Sent']}

df = pd.DataFrame.from_dict(df_dict)



100%|███████████████████████████████████████████| 39/39 [00:03<00:00, 11.47it/s]


In [20]:
df.iloc[20]

Original      Director on making beloved children's book cha...
O_Vader                                                  0.5106
O_TextBlob                          (0.09999999999999998, 0.95)
O_Flair                                               -0.976635
Processed     director making beloved children book characte...
P_Vader                                                  0.5106
P_TextBlob                          (0.09999999999999998, 0.95)
P_Flair                                                0.937315
Name: 20, dtype: object

In [105]:
len(articles_cnn[14].split())

0

In [15]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

text_original_classified = flair.data.Sentence(df['Original'][20])
flair_sentiment.predict(text_original_classified)
sentiment_o = text_original_classified.labels
print(sentiment_o)

print(text_original_classified.score)

['Sentence[11]: "Director on making beloved children's book character into bloodthirsty monster"'/'NEGATIVE' (0.9766)]
0.9766347408294678


In [143]:
print(str(text_original_classified.labels[0]))

Sentence[3


In [154]:
if re.findall('negative', str(text_original_classified.labels[0]), re.IGNORECASE):
    print('not cool')
else:
    print('fixe')

fixe
